# Module 6: Anomaly Detection
## Identifying Red-Flag Patients for Safety and Workflow Management

---

### Why Anomaly Detection Matters in ED

| Anomaly Type | Patient Safety Risk | Workflow Impact |
|--------------|--------------------|-----------------|
| **Extremely long waits** | Deterioration without assessment | Bottleneck indicator |
| **Skipped process steps** | Incomplete care, liability | Protocol breakdown |
| **Unusual sequences** | Errors, system issues | Training gaps |
| **High-acuity + long wait** | Life-threatening delay | Priority inversion |

### What We Detect

1. **Statistical Outliers** — Wait times beyond normal bounds
2. **Sequence Anomalies** — Events in wrong order or missing
3. **Priority Inversions** — Low-acuity seen before high-acuity
4. **Combined Risk Flags** — Multiple anomaly indicators

---

In [3]:
# =============================================================================
# CELL 1: IMPORTS
# =============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

print("✓ Imports complete")

✓ Imports complete


In [4]:
# =============================================================================
# CELL 2: LOAD DATA
# =============================================================================

%run ../utils/helpers.ipynb
%run data_loader.ipynb

filepath = "/Users/ishaandawra/Desktop/Machine Learning Notes/Machine Learning Projects/Analytics_Colloquia_Project/data/event_log_ED_MMA_2026.csv"
event_log, visits = load_and_prepare_data(filepath)

print(f"\nLoaded {len(event_log):,} events for {len(visits):,} visits")

NTH-ED DATA LOADING PIPELINE
📥 Loading event log...
   ✓ Loaded 90,965 events
   ✓ 16,011 unique patient visits
   ✓ Columns: ['visit_id', 'patient_id', 'initial_zone', 'age', 'month', 'day', 'gender', 'triage_code', 'triage_desc', 'disposition_code', 'disposition_desc', 'consult_desc', 'cdu_flag', 'consult_req_flag', 'consult_arrival_flag', 'event', 'timestamp']

⏰ Parsing timestamps...
   ✓ All timestamps parsed successfully
   ✓ Date range: 2021-03-31 23:59:00 to 2021-06-01 17:16:00

🔧 Handling missing data...
   • initial_zone: 1,950 missing (2.1%)
   • triage_code: 3 missing (0.0%)
   • consult_desc: 69,698 missing (76.6%)
   • age: 0 missing (0.0%)
   ✓ Missing zones filled with 'Unknown'
   ✓ Missing consults marked as 'No Consult'

📋 Standardizing columns for process mining...
   ✓ Created process mining columns: case_id, activity, resource
   ✓ Created outcome flags: is_admitted, is_lwbs

🔀 Sorting events with logical ordering...
   ✓ Events sorted by case_id → timestamp → log

---
## Section 1: Statistical Outlier Detection (Wait Times)

**Method:** Flag patients with wait times beyond statistical thresholds.

---

In [5]:
# =============================================================================
# CELL 3: DEFINE WAIT TIME THRESHOLDS
# =============================================================================

# CTAS Target Times (Canadian Triage Guidelines)
CTAS_TARGETS = {
    1: 0,     # Resuscitation: Immediate
    2: 15,    # Emergent: 15 minutes
    3: 30,    # Urgent: 30 minutes
    4: 60,    # Less Urgent: 60 minutes
    5: 120    # Non-Urgent: 120 minutes
}

# Statistical thresholds
PERCENTILE_THRESHOLDS = {
    'warning': 90,   # 90th percentile = warning
    'critical': 95,  # 95th percentile = critical
    'extreme': 99    # 99th percentile = extreme
}

print("CTAS Target Times (minutes):")
for ctas, target in CTAS_TARGETS.items():
    print(f"  CTAS {ctas}: {target} min")

CTAS Target Times (minutes):
  CTAS 1: 0 min
  CTAS 2: 15 min
  CTAS 3: 30 min
  CTAS 4: 60 min
  CTAS 5: 120 min


In [6]:
# =============================================================================
# CELL 4: CALCULATE WAIT TIME PERCENTILES
# =============================================================================

def calculate_wait_thresholds(visits_df: pd.DataFrame) -> dict:
    """
    Calculate wait time thresholds for anomaly detection.
    
    Returns thresholds by:
    - Overall (all patients)
    - By CTAS level
    - By Zone
    """
    
    thresholds = {}
    
    # Overall PIA thresholds
    pia_valid = visits_df['pia_minutes'].dropna()
    thresholds['overall'] = {
        'median': pia_valid.median(),
        'p90': pia_valid.quantile(0.90),
        'p95': pia_valid.quantile(0.95),
        'p99': pia_valid.quantile(0.99),
        'max': pia_valid.max()
    }
    
    # By CTAS level
    thresholds['by_ctas'] = {}
    for ctas in [1, 2, 3, 4, 5]:
        ctas_data = visits_df[visits_df['triage_code'] == ctas]['pia_minutes'].dropna()
        if len(ctas_data) > 0:
            thresholds['by_ctas'][ctas] = {
                'median': ctas_data.median(),
                'p90': ctas_data.quantile(0.90),
                'p95': ctas_data.quantile(0.95),
                'target': CTAS_TARGETS.get(ctas, 60),
                'count': len(ctas_data)
            }
    
    # By Zone
    thresholds['by_zone'] = {}
    for zone in visits_df['initial_zone'].dropna().unique():
        zone_data = visits_df[visits_df['initial_zone'] == zone]['pia_minutes'].dropna()
        if len(zone_data) > 10:
            thresholds['by_zone'][zone] = {
                'median': zone_data.median(),
                'p90': zone_data.quantile(0.90),
                'p95': zone_data.quantile(0.95)
            }
    
    return thresholds

# Calculate thresholds
wait_thresholds = calculate_wait_thresholds(visits)

print("=" * 60)
print("WAIT TIME THRESHOLDS")
print("=" * 60)

print(f"\nOverall PIA:")
print(f"  Median: {wait_thresholds['overall']['median']:.0f} min")
print(f"  90th percentile: {wait_thresholds['overall']['p90']:.0f} min")
print(f"  95th percentile: {wait_thresholds['overall']['p95']:.0f} min")
print(f"  99th percentile: {wait_thresholds['overall']['p99']:.0f} min")

print(f"\nBy CTAS Level:")
for ctas, stats in wait_thresholds['by_ctas'].items():
    print(f"  CTAS {ctas}: Median={stats['median']:.0f}min, P95={stats['p95']:.0f}min, Target={stats['target']}min")

WAIT TIME THRESHOLDS

Overall PIA:
  Median: 34 min
  90th percentile: 106 min
  95th percentile: 132 min
  99th percentile: 190 min

By CTAS Level:
  CTAS 1: Median=6min, P95=28min, Target=0min
  CTAS 2: Median=33min, P95=135min, Target=15min
  CTAS 3: Median=39min, P95=135min, Target=30min
  CTAS 4: Median=29min, P95=113min, Target=60min
  CTAS 5: Median=21min, P95=87min, Target=120min


In [7]:
# =============================================================================
# CELL 5: FLAG WAIT TIME ANOMALIES
# =============================================================================

def flag_wait_time_anomalies(visits_df: pd.DataFrame, thresholds: dict) -> pd.DataFrame:
    """
    Flag patients with anomalous wait times.
    
    Flags:
    - EXTREME: > 99th percentile
    - CRITICAL: > 95th percentile OR > 2x CTAS target
    - WARNING: > 90th percentile OR > CTAS target
    """
    
    df = visits_df.copy()
    
    # Initialize flags
    df['wait_anomaly'] = 'NORMAL'
    df['wait_anomaly_reason'] = ''
    
    # Overall percentile flags
    p90 = thresholds['overall']['p90']
    p95 = thresholds['overall']['p95']
    p99 = thresholds['overall']['p99']
    
    # Flag by percentile
    df.loc[df['pia_minutes'] > p90, 'wait_anomaly'] = 'WARNING'
    df.loc[df['pia_minutes'] > p90, 'wait_anomaly_reason'] = f'PIA > 90th percentile ({p90:.0f} min)'
    
    df.loc[df['pia_minutes'] > p95, 'wait_anomaly'] = 'CRITICAL'
    df.loc[df['pia_minutes'] > p95, 'wait_anomaly_reason'] = f'PIA > 95th percentile ({p95:.0f} min)'
    
    df.loc[df['pia_minutes'] > p99, 'wait_anomaly'] = 'EXTREME'
    df.loc[df['pia_minutes'] > p99, 'wait_anomaly_reason'] = f'PIA > 99th percentile ({p99:.0f} min)'
    
    # Flag by CTAS target breach (more clinically relevant)
    for ctas, target in CTAS_TARGETS.items():
        if target > 0:  # Skip CTAS 1 (immediate)
            # > 2x target is CRITICAL
            mask = (df['triage_code'] == ctas) & (df['pia_minutes'] > target * 2)
            df.loc[mask & (df['wait_anomaly'] != 'EXTREME'), 'wait_anomaly'] = 'CRITICAL'
            df.loc[mask, 'wait_anomaly_reason'] = f'CTAS {ctas}: PIA > 2x target ({target*2} min)'
            
            # > target is at least WARNING
            mask = (df['triage_code'] == ctas) & (df['pia_minutes'] > target) & (df['wait_anomaly'] == 'NORMAL')
            df.loc[mask, 'wait_anomaly'] = 'WARNING'
            df.loc[mask, 'wait_anomaly_reason'] = f'CTAS {ctas}: PIA > target ({target} min)'
    
    # Special case: CTAS 1-2 with ANY significant wait
    mask = (df['triage_code'] <= 2) & (df['pia_minutes'] > 30)
    df.loc[mask & (df['wait_anomaly'] == 'NORMAL'), 'wait_anomaly'] = 'CRITICAL'
    df.loc[mask & (df['wait_anomaly'] == 'WARNING'), 'wait_anomaly'] = 'CRITICAL'
    df.loc[mask, 'wait_anomaly_reason'] = 'High acuity (CTAS 1-2) with PIA > 30 min'
    
    return df

# Apply flags
visits_flagged = flag_wait_time_anomalies(visits, wait_thresholds)

# Summary
print("=" * 60)
print("WAIT TIME ANOMALY FLAGS")
print("=" * 60)

anomaly_counts = visits_flagged['wait_anomaly'].value_counts()
total = len(visits_flagged)

for level in ['EXTREME', 'CRITICAL', 'WARNING', 'NORMAL']:
    count = anomaly_counts.get(level, 0)
    pct = count / total * 100
    emoji = {'EXTREME': '🔴', 'CRITICAL': '🟠', 'WARNING': '🟡', 'NORMAL': '🟢'}[level]
    print(f"  {emoji} {level}: {count:,} ({pct:.1f}%)")

WAIT TIME ANOMALY FLAGS
  🔴 EXTREME: 154 (1.0%)
  🟠 CRITICAL: 5,237 (32.7%)
  🟡 WARNING: 3,526 (22.0%)
  🟢 NORMAL: 7,094 (44.3%)


---
## Section 2: Sequence Anomaly Detection

**Expected ED Process:**
```
Arrival → Triage → Registration → Assessment → [Treatment] → Discharge/Admission
```

**Anomalies to Detect:**
- Skipped steps (e.g., Assessment without Triage)
- Wrong order (e.g., Discharge before Assessment)
- Missing critical events

---

In [8]:
# =============================================================================
# CELL 6: DEFINE EXPECTED PROCESS RULES
# =============================================================================

# Expected event sequence (order matters)
EXPECTED_SEQUENCE = [
    'Triage',
    'Registration', 
    'Assessment',
    'Discharge'  # or Admission via Left ED
]

# Events that MUST occur before others
PRECEDENCE_RULES = {
    'Assessment': ['Triage'],           # Must have Triage before Assessment
    'Discharge': ['Triage', 'Registration'],  # Must have both before Discharge
    'Left ED': ['Triage'],              # Must have Triage before leaving
    'Consult Arrival': ['Consult Request'],  # Must request before arrival
}

# Events that should NOT be skipped for complete care
REQUIRED_EVENTS = ['Triage', 'Registration', 'Assessment']

# Valid terminal events
TERMINAL_EVENTS = ['Discharge', 'Left ED']

print("Process Rules Defined:")
print(f"  Expected sequence: {' → '.join(EXPECTED_SEQUENCE)}")
print(f"  Required events: {REQUIRED_EVENTS}")
print(f"  Precedence rules: {len(PRECEDENCE_RULES)} rules")

Process Rules Defined:
  Expected sequence: Triage → Registration → Assessment → Discharge
  Required events: ['Triage', 'Registration', 'Assessment']
  Precedence rules: 4 rules


In [9]:
# =============================================================================
# CELL 7: DETECT SEQUENCE ANOMALIES
# =============================================================================

def detect_sequence_anomalies(event_log: pd.DataFrame) -> pd.DataFrame:
    """
    Detect sequence anomalies in patient journeys.
    
    Returns DataFrame with one row per visit and anomaly details.
    """
    
    # Sort events by time within each visit
    event_log_sorted = event_log.sort_values(['case_id', 'timestamp'])
    
    anomalies = []
    
    for case_id, group in event_log_sorted.groupby('case_id'):
        events = group['event'].tolist()
        timestamps = group['timestamp'].tolist()
        
        case_anomalies = []
        
        # Check 1: Precedence violations
        event_positions = {e: i for i, e in enumerate(events)}
        
        for event, required_before in PRECEDENCE_RULES.items():
            if event in event_positions:
                event_pos = event_positions[event]
                for required in required_before:
                    if required not in event_positions:
                        case_anomalies.append({
                            'type': 'MISSING_PRECEDENT',
                            'severity': 'CRITICAL',
                            'detail': f'{event} without {required}'
                        })
                    elif event_positions[required] > event_pos:
                        case_anomalies.append({
                            'type': 'WRONG_ORDER',
                            'severity': 'CRITICAL',
                            'detail': f'{event} before {required}'
                        })
        
        # Check 2: Missing required events
        for required in REQUIRED_EVENTS:
            if required not in events:
                # Check if patient completed their visit (has terminal event)
                has_terminal = any(e in TERMINAL_EVENTS for e in events)
                if has_terminal:
                    case_anomalies.append({
                        'type': 'SKIPPED_STEP',
                        'severity': 'WARNING',
                        'detail': f'Missing {required}'
                    })
        
        # Check 3: Discharge before Assessment
        if 'Discharge' in events and 'Assessment' not in events:
            case_anomalies.append({
                'type': 'INCOMPLETE_CARE',
                'severity': 'CRITICAL',
                'detail': 'Discharge without Assessment'
            })
        
        # Check 4: Assessment before Registration (unusual but not critical)
        if 'Assessment' in event_positions and 'Registration' in event_positions:
            if event_positions['Assessment'] < event_positions['Registration']:
                case_anomalies.append({
                    'type': 'UNUSUAL_ORDER',
                    'severity': 'WARNING',
                    'detail': 'Assessment before Registration'
                })
        
        # Check 5: Multiple terminal events (data quality issue)
        terminal_count = sum(1 for e in events if e in TERMINAL_EVENTS)
        if terminal_count > 1:
            case_anomalies.append({
                'type': 'DATA_QUALITY',
                'severity': 'WARNING',
                'detail': f'Multiple terminal events ({terminal_count})'
            })
        
        # Store results
        if case_anomalies:
            worst_severity = 'CRITICAL' if any(a['severity'] == 'CRITICAL' for a in case_anomalies) else 'WARNING'
            anomalies.append({
                'case_id': case_id,
                'sequence_anomaly': worst_severity,
                'anomaly_count': len(case_anomalies),
                'anomaly_types': ', '.join(set(a['type'] for a in case_anomalies)),
                'anomaly_details': '; '.join(a['detail'] for a in case_anomalies),
                'event_sequence': ' → '.join(events)
            })
        else:
            anomalies.append({
                'case_id': case_id,
                'sequence_anomaly': 'NORMAL',
                'anomaly_count': 0,
                'anomaly_types': '',
                'anomaly_details': '',
                'event_sequence': ' → '.join(events)
            })
    
    return pd.DataFrame(anomalies)

# Detect sequence anomalies
print("Detecting sequence anomalies...")
sequence_anomalies = detect_sequence_anomalies(event_log)

# Summary
print("\n" + "=" * 60)
print("SEQUENCE ANOMALY DETECTION")
print("=" * 60)

seq_counts = sequence_anomalies['sequence_anomaly'].value_counts()
total = len(sequence_anomalies)

for level in ['CRITICAL', 'WARNING', 'NORMAL']:
    count = seq_counts.get(level, 0)
    pct = count / total * 100
    emoji = {'CRITICAL': '🔴', 'WARNING': '🟡', 'NORMAL': '🟢'}[level]
    print(f"  {emoji} {level}: {count:,} ({pct:.1f}%)")

# Top anomaly types
print("\nTop Anomaly Types:")
anomaly_types = sequence_anomalies[sequence_anomalies['anomaly_types'] != '']['anomaly_types'].str.split(', ').explode().value_counts()
for atype, count in anomaly_types.head(5).items():
    print(f"  • {atype}: {count:,}")

Detecting sequence anomalies...

SEQUENCE ANOMALY DETECTION
  🔴 CRITICAL: 182 (1.1%)
  🟡 WARNING: 15,676 (97.9%)
  🟢 NORMAL: 153 (1.0%)

Top Anomaly Types:
  • DATA_QUALITY: 15,825
  • INCOMPLETE_CARE: 163
  • SKIPPED_STEP: 163
  • UNUSUAL_ORDER: 106
  • WRONG_ORDER: 20


---
## Section 3: Priority Inversion Detection

**Problem:** Lower-acuity patients seen before higher-acuity patients who arrived earlier.

**Why it matters:** Violates triage principles, patient safety risk.

---

In [10]:
# =============================================================================
# CELL 8: DETECT PRIORITY INVERSIONS
# =============================================================================

def detect_priority_inversions(visits_df: pd.DataFrame, event_log: pd.DataFrame) -> pd.DataFrame:
    """
    Detect cases where lower-acuity patients were seen before higher-acuity patients.
    
    Priority inversion = CTAS 4 patient gets Assessment before CTAS 2 patient
                         who arrived earlier.
    """
    
    # Get assessment times
    assessments = event_log[event_log['event'] == 'Assessment'][['case_id', 'timestamp']].copy()
    assessments.columns = ['case_id', 'assessment_time']
    
    # Merge with visit data
    df = visits_df[['case_id', 'triage_code', 'triage_time', 'initial_zone']].merge(
        assessments, on='case_id', how='inner'
    )
    
    # Sort by assessment time
    df = df.sort_values('assessment_time').reset_index(drop=True)
    
    inversions = []
    
    # For each assessed patient, check if any higher-acuity patient who arrived earlier
    # is still waiting (was assessed later)
    for i, row in df.iterrows():
        # Find higher-acuity patients who arrived earlier but assessed later
        earlier_arrivals = df[
            (df['triage_time'] < row['triage_time']) &  # Arrived earlier
            (df['assessment_time'] > row['assessment_time']) &  # Assessed later
            (df['triage_code'] < row['triage_code']) &  # Higher acuity (lower number)
            (df['initial_zone'] == row['initial_zone'])  # Same zone (fair comparison)
        ]
        
        if len(earlier_arrivals) > 0:
            worst_inversion = earlier_arrivals.iloc[0]
            acuity_diff = row['triage_code'] - worst_inversion['triage_code']
            
            inversions.append({
                'case_id': row['case_id'],
                'patient_ctas': row['triage_code'],
                'patient_zone': row['initial_zone'],
                'bypassed_case_id': worst_inversion['case_id'],
                'bypassed_ctas': worst_inversion['triage_code'],
                'acuity_difference': acuity_diff,
                'severity': 'CRITICAL' if acuity_diff >= 2 else 'WARNING'
            })
    
    return pd.DataFrame(inversions)

# Detect priority inversions
print("Detecting priority inversions...")
priority_inversions = detect_priority_inversions(visits, event_log)

print("\n" + "=" * 60)
print("PRIORITY INVERSION DETECTION")
print("=" * 60)

if len(priority_inversions) > 0:
    print(f"\nTotal inversions detected: {len(priority_inversions):,}")
    
    severity_counts = priority_inversions['severity'].value_counts()
    print(f"  🔴 CRITICAL (≥2 CTAS levels): {severity_counts.get('CRITICAL', 0):,}")
    print(f"  🟡 WARNING (1 CTAS level): {severity_counts.get('WARNING', 0):,}")
    
    print("\nBy Zone:")
    zone_counts = priority_inversions.groupby('patient_zone').size().sort_values(ascending=False)
    for zone, count in zone_counts.head(5).items():
        print(f"  • {zone}: {count:,}")
else:
    print("\nNo priority inversions detected.")

Detecting priority inversions...

PRIORITY INVERSION DETECTION

Total inversions detected: 1,041
  🔴 CRITICAL (≥2 CTAS levels): 149
  🟡 WARNING (1 CTAS level): 892

By Zone:
  • GZ: 426
  • YZ: 258
  • A: 206
  • EPZ: 120
  • SA: 27


---
## Section 4: Combined Anomaly Scoring

**Goal:** Create a single "Red Flag" score combining all anomaly types.

---

In [11]:
# =============================================================================
# CELL 9: COMBINE ALL ANOMALY FLAGS
# =============================================================================

def create_combined_anomaly_score(visits_df: pd.DataFrame, 
                                   sequence_df: pd.DataFrame,
                                   inversions_df: pd.DataFrame) -> pd.DataFrame:
    """
    Combine all anomaly flags into a single Red Flag score.
    
    Scoring:
    - Wait time EXTREME: +3 points
    - Wait time CRITICAL: +2 points
    - Wait time WARNING: +1 point
    - Sequence CRITICAL: +2 points
    - Sequence WARNING: +1 point
    - Priority inversion: +1-2 points
    
    Red Flag Levels:
    - ≥5 points: EXTREME
    - 3-4 points: CRITICAL
    - 1-2 points: WARNING
    - 0 points: NORMAL
    """
    
    # Start with visits data
    df = visits_df[['case_id', 'triage_code', 'initial_zone', 'pia_minutes', 
                    'wait_anomaly', 'wait_anomaly_reason']].copy()
    
    # Merge sequence anomalies
    df = df.merge(
        sequence_df[['case_id', 'sequence_anomaly', 'anomaly_details']],
        on='case_id',
        how='left'
    )
    df['sequence_anomaly'] = df['sequence_anomaly'].fillna('NORMAL')
    
    # Flag priority inversions
    if len(inversions_df) > 0:
        inversion_cases = inversions_df.groupby('bypassed_case_id')['severity'].first().reset_index()
        inversion_cases.columns = ['case_id', 'inversion_severity']
        df = df.merge(inversion_cases, on='case_id', how='left')
    else:
        df['inversion_severity'] = None
    
    # Calculate anomaly score
    df['anomaly_score'] = 0
    
    # Wait time points
    df.loc[df['wait_anomaly'] == 'EXTREME', 'anomaly_score'] += 3
    df.loc[df['wait_anomaly'] == 'CRITICAL', 'anomaly_score'] += 2
    df.loc[df['wait_anomaly'] == 'WARNING', 'anomaly_score'] += 1
    
    # Sequence points
    df.loc[df['sequence_anomaly'] == 'CRITICAL', 'anomaly_score'] += 2
    df.loc[df['sequence_anomaly'] == 'WARNING', 'anomaly_score'] += 1
    
    # Inversion points
    df.loc[df['inversion_severity'] == 'CRITICAL', 'anomaly_score'] += 2
    df.loc[df['inversion_severity'] == 'WARNING', 'anomaly_score'] += 1
    
    # Assign Red Flag level
    df['red_flag'] = 'NORMAL'
    df.loc[df['anomaly_score'] >= 1, 'red_flag'] = 'WARNING'
    df.loc[df['anomaly_score'] >= 3, 'red_flag'] = 'CRITICAL'
    df.loc[df['anomaly_score'] >= 5, 'red_flag'] = 'EXTREME'
    
    # Generate summary reason
    def summarize_reasons(row):
        reasons = []
        if row['wait_anomaly'] != 'NORMAL':
            reasons.append(f"Wait: {row['wait_anomaly']}")
        if row['sequence_anomaly'] != 'NORMAL':
            reasons.append(f"Sequence: {row['sequence_anomaly']}")
        if pd.notna(row['inversion_severity']):
            reasons.append(f"Bypassed: {row['inversion_severity']}")
        return '; '.join(reasons) if reasons else 'No anomalies'
    
    df['red_flag_reasons'] = df.apply(summarize_reasons, axis=1)
    
    return df

# Create combined anomaly scores
anomaly_summary = create_combined_anomaly_score(visits_flagged, sequence_anomalies, priority_inversions)

print("=" * 60)
print("COMBINED RED FLAG SUMMARY")
print("=" * 60)

flag_counts = anomaly_summary['red_flag'].value_counts()
total = len(anomaly_summary)

for level in ['EXTREME', 'CRITICAL', 'WARNING', 'NORMAL']:
    count = flag_counts.get(level, 0)
    pct = count / total * 100
    emoji = {'EXTREME': '🔴', 'CRITICAL': '🟠', 'WARNING': '🟡', 'NORMAL': '🟢'}[level]
    print(f"  {emoji} {level}: {count:,} ({pct:.1f}%)")

print(f"\nTotal Red Flags (WARNING+): {(anomaly_summary['red_flag'] != 'NORMAL').sum():,}")

COMBINED RED FLAG SUMMARY
  🔴 EXTREME: 98 (0.6%)
  🟠 CRITICAL: 5,407 (33.8%)
  🟡 WARNING: 10,411 (65.0%)
  🟢 NORMAL: 95 (0.6%)

Total Red Flags (WARNING+): 15,916


---
## Section 5: Anomaly Visualization

---

In [12]:
# =============================================================================
# CELL 10: RED FLAG DISTRIBUTION VISUALIZATION
# =============================================================================

def plot_anomaly_distribution(anomaly_df: pd.DataFrame):
    """Create comprehensive anomaly distribution visualization."""
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Red Flag Distribution',
            'Red Flags by CTAS Level',
            'Red Flags by Zone',
            'Wait Time vs Anomaly Score'
        ),
        specs=[[{"type": "pie"}, {"type": "bar"}],
               [{"type": "bar"}, {"type": "scatter"}]]
    )
    
    colors = {'EXTREME': '#DC2626', 'CRITICAL': '#F97316', 'WARNING': '#FBBF24', 'NORMAL': '#22C55E'}
    
    # 1. Pie chart of red flag distribution
    flag_counts = anomaly_df['red_flag'].value_counts()
    fig.add_trace(
        go.Pie(
            labels=flag_counts.index,
            values=flag_counts.values,
            marker_colors=[colors[x] for x in flag_counts.index],
            hole=0.4
        ),
        row=1, col=1
    )
    
    # 2. Red flags by CTAS
    ctas_flags = anomaly_df[anomaly_df['red_flag'] != 'NORMAL'].groupby('triage_code').size()
    fig.add_trace(
        go.Bar(
            x=[f'CTAS {int(x)}' for x in ctas_flags.index],
            y=ctas_flags.values,
            marker_color='#F97316'
        ),
        row=1, col=2
    )
    
    # 3. Red flags by Zone
    zone_flags = anomaly_df[anomaly_df['red_flag'] != 'NORMAL'].groupby('initial_zone').size().sort_values(ascending=True)
    fig.add_trace(
        go.Bar(
            x=zone_flags.values,
            y=zone_flags.index,
            orientation='h',
            marker_color='#F97316'
        ),
        row=2, col=1
    )
    
    # 4. Wait time vs anomaly score scatter
    sample = anomaly_df.sample(min(1000, len(anomaly_df)), random_state=42)
    fig.add_trace(
        go.Scatter(
            x=sample['pia_minutes'],
            y=sample['anomaly_score'],
            mode='markers',
            marker=dict(
                color=[colors[x] for x in sample['red_flag']],
                size=6,
                opacity=0.6
            )
        ),
        row=2, col=2
    )
    
    fig.update_layout(
        height=600,
        showlegend=False,
        title_text="<b>Anomaly Detection Dashboard</b>"
    )
    
    fig.update_xaxes(title_text="PIA (minutes)", row=2, col=2)
    fig.update_yaxes(title_text="Anomaly Score", row=2, col=2)
    
    fig.show()

# Plot
plot_anomaly_distribution(anomaly_summary)

In [13]:
# =============================================================================
# CELL 11: RED FLAG CASES TABLE
# =============================================================================

def display_red_flag_cases(anomaly_df: pd.DataFrame, max_display: int = 20):
    """Display table of highest-priority red flag cases."""
    
    # Get worst cases
    red_flags = anomaly_df[anomaly_df['red_flag'] != 'NORMAL'].sort_values(
        'anomaly_score', ascending=False
    ).head(max_display)
    
    print("=" * 80)
    print("TOP RED FLAG CASES (Highest Anomaly Scores)")
    print("=" * 80)
    
    display_cols = ['case_id', 'triage_code', 'initial_zone', 'pia_minutes', 
                    'anomaly_score', 'red_flag', 'red_flag_reasons']
    
    display_df = red_flags[display_cols].copy()
    display_df.columns = ['Case ID', 'CTAS', 'Zone', 'PIA (min)', 'Score', 'Flag', 'Reasons']
    
    return display_df

# Display top red flags
top_red_flags = display_red_flag_cases(anomaly_summary)
display(top_red_flags)

TOP RED FLAG CASES (Highest Anomaly Scores)


,Case ID,CTAS,Zone,PIA (min),Score,Flag,Reasons
14240,7466544,2.0,YZ,199.0,6,EXTREME,Wait: EXTREME; Sequence: WARNING; Bypassed: CR...
7513,7431882,2.0,YZ,509.0,5,EXTREME,Wait: EXTREME; Sequence: WARNING; Bypassed: WA...
6283,7424004,2.0,GZ,58.0,5,EXTREME,Wait: CRITICAL; Sequence: WARNING; Bypassed: C...
10733,7449822,2.0,A,197.0,5,EXTREME,Wait: EXTREME; Sequence: WARNING; Bypassed: WA...
11879,7455411,2.0,GZ,51.0,5,EXTREME,Wait: CRITICAL; Sequence: WARNING; Bypassed: C...
15279,7472093,2.0,GZ,68.0,5,EXTREME,Wait: CRITICAL; Sequence: WARNING; Bypassed: C...
7383,7431422,2.0,GZ,56.0,5,EXTREME,Wait: CRITICAL; Sequence: WARNING; Bypassed: C...
2950,7402209,2.0,A,193.0,5,EXTREME,Wait: EXTREME; Sequence: WARNING; Bypassed: WA...
6290,7424078,3.0,GZ,218.0,5,EXTREME,Wait: EXTREME; Sequence: WARNING; Bypassed: WA...
3862,7408513,2.0,GZ,79.0,5,EXTREME,Wait: CRITICAL; Sequence: WARNING; Bypassed: C...


In [14]:
# =============================================================================
# CELL 12: ANOMALY HEATMAP BY ZONE AND HOUR
# =============================================================================

def plot_anomaly_heatmap(anomaly_df: pd.DataFrame, visits_df: pd.DataFrame):
    """Plot heatmap showing when and where anomalies occur."""
    
    # Merge with arrival hour
    df = anomaly_df.merge(
        visits_df[['case_id', 'arrival_hour']],
        on='case_id',
        how='left'
    )
    
    # Calculate anomaly rate by zone and hour
    df['is_anomaly'] = (df['red_flag'] != 'NORMAL').astype(int)
    
    heatmap_data = df.groupby(['initial_zone', 'arrival_hour'])['is_anomaly'].mean().unstack()
    
    # Plot
    fig = px.imshow(
        heatmap_data,
        labels=dict(x="Hour of Day", y="Zone", color="Anomaly Rate"),
        title="<b>Anomaly Rate by Zone and Hour</b><br><sup>Darker = More anomalies</sup>",
        color_continuous_scale='Reds',
        aspect='auto'
    )
    
    fig.update_layout(height=400)
    fig.show()
    
    # Find hotspots
    print("\n🔥 ANOMALY HOTSPOTS (Top 5 Zone-Hour combinations):")
    hotspots = df.groupby(['initial_zone', 'arrival_hour']).agg({
        'is_anomaly': ['sum', 'mean', 'count']
    }).reset_index()
    hotspots.columns = ['Zone', 'Hour', 'Anomaly Count', 'Anomaly Rate', 'Total']
    hotspots = hotspots[hotspots['Total'] >= 50].sort_values('Anomaly Rate', ascending=False).head(5)
    
    for _, row in hotspots.iterrows():
        print(f"  • {row['Zone']} at {int(row['Hour'])}:00 — {row['Anomaly Rate']:.1%} anomaly rate ({int(row['Anomaly Count'])} cases)")

# Plot heatmap
plot_anomaly_heatmap(anomaly_summary, visits)


🔥 ANOMALY HOTSPOTS (Top 5 Zone-Hour combinations):
  • EPZ at 12:00 — 100.0% anomaly rate (170 cases)
  • YZ at 8:00 — 100.0% anomaly rate (151 cases)
  • EPZ at 13:00 — 100.0% anomaly rate (144 cases)
  • YZ at 6:00 — 100.0% anomaly rate (66 cases)
  • YZ at 5:00 — 100.0% anomaly rate (86 cases)


Almost every zone has 60-100% anomaly rates. Most "anomalies" are CTAS target breaches — patients waiting longer than guidelines recommend. 

- Yellow Zone is the main bottleneck
- EPZ at 12:00-13:00 = 100% anomaly (Psych zone overwhelmed at lunch — need coverage)

---
## Section 6: Anomaly Detection Functions for Dashboard

---

In [15]:
# =============================================================================
# CELL 13: REAL-TIME ANOMALY DETECTOR CLASS
# =============================================================================

class AnomalyDetector:
    """
    Real-time anomaly detection for ED triage dashboard.
    
    Usage:
        detector = AnomalyDetector(historical_visits)
        flags = detector.check_patient(patient_data)
    """
    
    def __init__(self, historical_data: pd.DataFrame):
        """Initialize with historical data for threshold calculation."""
        self.thresholds = calculate_wait_thresholds(historical_data)
        self.ctas_targets = CTAS_TARGETS
    
    def check_wait_time(self, pia_minutes: float, ctas: int) -> dict:
        """
        Check if wait time is anomalous.
        
        Returns dict with flag level and reason.
        """
        
        if pd.isna(pia_minutes):
            return {'flag': 'UNKNOWN', 'reason': 'PIA not recorded'}
        
        p90 = self.thresholds['overall']['p90']
        p95 = self.thresholds['overall']['p95']
        p99 = self.thresholds['overall']['p99']
        target = self.ctas_targets.get(ctas, 60)
        
        # Check percentile thresholds
        if pia_minutes > p99:
            return {'flag': 'EXTREME', 'reason': f'PIA ({pia_minutes:.0f}min) > 99th percentile ({p99:.0f}min)'}
        
        if pia_minutes > p95:
            return {'flag': 'CRITICAL', 'reason': f'PIA ({pia_minutes:.0f}min) > 95th percentile ({p95:.0f}min)'}
        
        # Check CTAS target
        if ctas <= 2 and pia_minutes > 30:
            return {'flag': 'CRITICAL', 'reason': f'High acuity (CTAS {ctas}) with PIA > 30 min'}
        
        if target > 0 and pia_minutes > target * 2:
            return {'flag': 'CRITICAL', 'reason': f'PIA > 2x CTAS {ctas} target ({target*2}min)'}
        
        if pia_minutes > p90:
            return {'flag': 'WARNING', 'reason': f'PIA ({pia_minutes:.0f}min) > 90th percentile ({p90:.0f}min)'}
        
        if target > 0 and pia_minutes > target:
            return {'flag': 'WARNING', 'reason': f'PIA > CTAS {ctas} target ({target}min)'}
        
        return {'flag': 'NORMAL', 'reason': 'Within normal bounds'}
    
    def check_sequence(self, events: list) -> dict:
        """
        Check if event sequence has anomalies.
        
        Parameters:
        -----------
        events: List of event names in chronological order
        """
        
        anomalies = []
        
        # Check precedence rules
        event_positions = {e: i for i, e in enumerate(events)}
        
        for event, required_before in PRECEDENCE_RULES.items():
            if event in event_positions:
                for required in required_before:
                    if required not in event_positions:
                        anomalies.append(f'{event} without {required}')
                    elif event_positions[required] > event_positions[event]:
                        anomalies.append(f'{event} before {required}')
        
        # Check missing required events
        has_terminal = any(e in TERMINAL_EVENTS for e in events)
        if has_terminal:
            for required in REQUIRED_EVENTS:
                if required not in events:
                    anomalies.append(f'Missing {required}')
        
        if anomalies:
            severity = 'CRITICAL' if any('without' in a or 'before' in a for a in anomalies) else 'WARNING'
            return {'flag': severity, 'reason': '; '.join(anomalies)}
        
        return {'flag': 'NORMAL', 'reason': 'Valid sequence'}
    
    def get_combined_flag(self, wait_flag: dict, sequence_flag: dict) -> dict:
        """
        Combine wait time and sequence flags into overall red flag.
        """
        
        score = 0
        reasons = []
        
        # Score wait flag
        if wait_flag['flag'] == 'EXTREME':
            score += 3
            reasons.append(f"Wait: {wait_flag['reason']}")
        elif wait_flag['flag'] == 'CRITICAL':
            score += 2
            reasons.append(f"Wait: {wait_flag['reason']}")
        elif wait_flag['flag'] == 'WARNING':
            score += 1
            reasons.append(f"Wait: {wait_flag['reason']}")
        
        # Score sequence flag
        if sequence_flag['flag'] == 'CRITICAL':
            score += 2
            reasons.append(f"Sequence: {sequence_flag['reason']}")
        elif sequence_flag['flag'] == 'WARNING':
            score += 1
            reasons.append(f"Sequence: {sequence_flag['reason']}")
        
        # Determine overall flag
        if score >= 5:
            flag = 'EXTREME'
        elif score >= 3:
            flag = 'CRITICAL'
        elif score >= 1:
            flag = 'WARNING'
        else:
            flag = 'NORMAL'
        
        return {
            'flag': flag,
            'score': score,
            'reasons': reasons,
            'summary': '; '.join(reasons) if reasons else 'No anomalies detected'
        }

# Initialize detector
detector = AnomalyDetector(visits)
print("✓ AnomalyDetector initialized")

✓ AnomalyDetector initialized


In [16]:
# =============================================================================
# CELL 14: TEST ANOMALY DETECTOR
# =============================================================================

print("=" * 60)
print("ANOMALY DETECTOR TEST CASES")
print("=" * 60)

test_cases = [
    {'pia': 25, 'ctas': 3, 'events': ['Triage', 'Registration', 'Assessment', 'Discharge']},
    {'pia': 120, 'ctas': 2, 'events': ['Triage', 'Registration', 'Assessment', 'Discharge']},
    {'pia': 45, 'ctas': 3, 'events': ['Assessment', 'Discharge']},  # Missing Triage
    {'pia': 200, 'ctas': 4, 'events': ['Triage', 'Discharge']},  # Missing Assessment + long wait
    {'pia': 15, 'ctas': 1, 'events': ['Triage', 'Assessment', 'Registration', 'Discharge']},  # Wrong order
]

for i, case in enumerate(test_cases, 1):
    wait_check = detector.check_wait_time(case['pia'], case['ctas'])
    seq_check = detector.check_sequence(case['events'])
    combined = detector.get_combined_flag(wait_check, seq_check)
    
    emoji = {'EXTREME': '🔴', 'CRITICAL': '🟠', 'WARNING': '🟡', 'NORMAL': '🟢'}[combined['flag']]
    
    print(f"\nCase {i}: CTAS {case['ctas']}, PIA={case['pia']}min")
    print(f"  Events: {' → '.join(case['events'])}")
    print(f"  {emoji} Result: {combined['flag']} (Score: {combined['score']})")
    if combined['reasons']:
        for reason in combined['reasons']:
            print(f"     • {reason}")

ANOMALY DETECTOR TEST CASES

Case 1: CTAS 3, PIA=25min
  Events: Triage → Registration → Assessment → Discharge
  🟢 Result: NORMAL (Score: 0)

Case 2: CTAS 2, PIA=120min
  Events: Triage → Registration → Assessment → Discharge
  🟡 Result: WARNING (Score: 2)
     • Wait: High acuity (CTAS 2) with PIA > 30 min

Case 3: CTAS 3, PIA=45min
  Events: Assessment → Discharge
  🟠 Result: CRITICAL (Score: 3)
     • Wait: PIA > CTAS 3 target (30min)
     • Sequence: Assessment without Triage; Discharge without Triage; Discharge without Registration; Missing Triage; Missing Registration

Case 4: CTAS 4, PIA=200min
  Events: Triage → Discharge
  🔴 Result: EXTREME (Score: 5)
     • Wait: PIA (200min) > 99th percentile (190min)
     • Sequence: Discharge without Registration; Missing Registration; Missing Assessment

Case 5: CTAS 1, PIA=15min
  Events: Triage → Assessment → Registration → Discharge
  🟢 Result: NORMAL (Score: 0)


In [17]:
# =============================================================================
# CELL 15: EXPORT ANOMALY DATA
# =============================================================================

# Store for dashboard use
ANOMALY_DATA = {
    'summary': anomaly_summary,
    'thresholds': wait_thresholds,
    'detector': detector,
    'sequence_anomalies': sequence_anomalies,
    'priority_inversions': priority_inversions
}

print("✓ Anomaly data exported to ANOMALY_DATA dictionary")

✓ Anomaly data exported to ANOMALY_DATA dictionary


---
## Section 7: Business Impact Summary

---

In [18]:
# =============================================================================
# CELL 16: BUSINESS IMPACT ANALYSIS
# =============================================================================

print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║                    ANOMALY DETECTION — BUSINESS IMPACT                       ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  WHY THIS MATTERS FOR PATIENT SAFETY                                         ║
║  ────────────────────────────────────                                        ║
║                                                                              ║
║  🔴 EXTREME WAIT TIMES                                                       ║
║     • Patient deterioration without assessment                               ║
║     • Missed critical diagnoses (MI, stroke, sepsis)                         ║
║     • Legal liability for delayed care                                       ║
║                                                                              ║
║  🔴 SKIPPED PROCESS STEPS                                                    ║
║     • Incomplete clinical assessment                                         ║
║     • Documentation gaps (liability)                                         ║
║     • Missed allergies, medications, history                                 ║
║                                                                              ║
║  🔴 PRIORITY INVERSIONS                                                      ║
║     • High-acuity patients waiting while low-acuity seen                     ║
║     • Triage system failure indicator                                        ║
║     • Potential adverse outcomes                                             ║
║                                                                              ║
║  WHY THIS MATTERS FOR WORKFLOW                                               ║
║  ──────────────────────────────────                                          ║
║                                                                              ║
║  📊 IDENTIFIES BOTTLENECKS                                                   ║
║     • Which zones have most anomalies?                                       ║
║     • Which hours are problematic?                                           ║
║     • Resource allocation decisions                                          ║
║                                                                              ║
║  📊 TRAINING OPPORTUNITIES                                                   ║
║     • Sequence anomalies reveal protocol gaps                                ║
║     • Staff education targets                                                ║
║                                                                              ║
║  📊 QUALITY IMPROVEMENT                                                      ║
║     • Track anomaly rates over time                                          ║
║     • Measure intervention effectiveness                                     ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
""")

# Quantify impact
total_visits = len(anomaly_summary)
critical_plus = (anomaly_summary['red_flag'].isin(['CRITICAL', 'EXTREME'])).sum()
warning_plus = (anomaly_summary['red_flag'] != 'NORMAL').sum()

print(f"\n📊 QUANTIFIED IMPACT:")
print(f"   • {critical_plus:,} patients ({critical_plus/total_visits*100:.1f}%) had CRITICAL+ anomalies")
print(f"   • {warning_plus:,} patients ({warning_plus/total_visits*100:.1f}%) had any anomaly")
print(f"   • These represent opportunities for intervention and improvement")


╔══════════════════════════════════════════════════════════════════════════════╗
║                    ANOMALY DETECTION — BUSINESS IMPACT                       ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  WHY THIS MATTERS FOR PATIENT SAFETY                                         ║
║  ────────────────────────────────────                                        ║
║                                                                              ║
║  🔴 EXTREME WAIT TIMES                                                       ║
║     • Patient deterioration without assessment                               ║
║     • Missed critical diagnoses (MI, stroke, sepsis)                         ║
║     • Legal liability for delayed care                                       ║
║                                                                              ║
║  🔴 SKIPPED PROCESS STEPS  

In [19]:
# =============================================================================
# CELL 17: MODULE SUMMARY
# =============================================================================

print("""
================================================================================
MODULE 6: ANOMALY DETECTION — COMPLETE
================================================================================

WHAT WE BUILT:
─────────────────────────────────────────────────────────────────────────────────

1. WAIT TIME ANOMALY DETECTION
   • Statistical thresholds (90th, 95th, 99th percentiles)
   • CTAS-specific target breaches
   • High-acuity special handling

2. SEQUENCE ANOMALY DETECTION
   • Missing required steps (Triage, Registration, Assessment)
   • Wrong event order (Assessment before Triage)
   • Precedence rule violations

3. PRIORITY INVERSION DETECTION
   • Lower-acuity patients seen before higher-acuity
   • Zone-specific analysis

4. COMBINED RED FLAG SCORING
   • Unified anomaly score (0-7 scale)
   • EXTREME / CRITICAL / WARNING / NORMAL levels

5. VISUALIZATIONS
   • Distribution charts
   • Zone-hour heatmaps
   • Red flag case tables

6. REAL-TIME DETECTOR CLASS
   • AnomalyDetector for dashboard integration
   • check_wait_time(), check_sequence(), get_combined_flag()

KEY OUTPUTS:
─────────────────────────────────────────────────────────────────────────────────
• ANOMALY_DATA dictionary with all results
• AnomalyDetector class for real-time use
• Visualizations for Triage Lead dashboard

================================================================================
""")


MODULE 6: ANOMALY DETECTION — COMPLETE

WHAT WE BUILT:
─────────────────────────────────────────────────────────────────────────────────

1. WAIT TIME ANOMALY DETECTION
   • Statistical thresholds (90th, 95th, 99th percentiles)
   • CTAS-specific target breaches
   • High-acuity special handling

2. SEQUENCE ANOMALY DETECTION
   • Missing required steps (Triage, Registration, Assessment)
   • Wrong event order (Assessment before Triage)
   • Precedence rule violations

3. PRIORITY INVERSION DETECTION
   • Lower-acuity patients seen before higher-acuity
   • Zone-specific analysis

4. COMBINED RED FLAG SCORING
   • Unified anomaly score (0-7 scale)
   • EXTREME / CRITICAL / WARNING / NORMAL levels

5. VISUALIZATIONS
   • Distribution charts
   • Zone-hour heatmaps
   • Red flag case tables

6. REAL-TIME DETECTOR CLASS
   • AnomalyDetector for dashboard integration
   • check_wait_time(), check_sequence(), get_combined_flag()

KEY OUTPUTS:
───────────────────────────────────────────────